In [5]:
import os
os.listdir()

['.git',
 'anomaly_detection.ipynb',
 'concepts.txt',
 'data',
 'PyEMD',
 'README.md']

In [119]:
import pandas as pd
datafolder = os.listdir('data/s3Files/')
# df = pd.concat([pd.read_csv(f, header=None,sep=',') for f in datafolder])
df = pd.read_csv('data/s3Files/data2.txt',header=None,sep=',')


In [120]:
df.head(2)

,0,1,2,3,4
0,NaN,2102,2318,97,NaN
1,NaN,2095,2326,94,NaN


In [121]:
df.drop(columns=[0,4],inplace=True)

In [122]:
df.columns = ['current_1', 'current_2', 'current_3']

Perform motor current signature analysis: You can use the scipy library to perform Fourier transforms on the current readings from each phase of the motor and plot the resulting frequency spectra. This will help you identify any anomalies or patterns in the spectra that may indicate a fault in the motor.

In [107]:
df

,current_1,current_2,current_3
0,2102,2318,97
1,2095,2326,94
2,2087,2336,78
3,1867,2429,283
4,1862,2430,297
...,...,...,...
9995,1809,2135,789
9996,1816,2136,790
9997,1824,2122,790
9998,1829,2121,791


In [108]:
df.loc[4] = 10000

In [109]:
df

,current_1,current_2,current_3
0,2102,2318,97
1,2095,2326,94
2,2087,2336,78
3,1867,2429,283
4,10000,10000,10000
...,...,...,...
9995,1809,2135,789
9996,1816,2136,790
9997,1824,2122,790
9998,1829,2121,791


In [110]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

In [111]:
scaled_df

array([[ 0.54407143,  0.68071122, -1.23440259],
       [ 0.50889197,  0.72149975, -1.24469526],
       [ 0.46868688,  0.77248542, -1.29958948],
       ...,
       [-0.85305563, -0.31860792,  1.14320339],
       [-0.82792745, -0.32370649,  1.14663428],
       [-0.79777362, -0.39508643,  1.16035784]])

In [112]:
from scipy.fft import fft

fft_data = fft(scaled_df, axis=1)


In [113]:
fft_data[0]

array([-0.00961995-0.j        ,  0.82091712-1.65853721j,
        0.82091712+1.65853721j])

Perform model-based VI analysis: To perform model-based VI analysis, you will need to create a mathematical model of the motor that takes into account both its electrical and mechanical behavior. This can be done using tools like MATLAB/Simulink or Python libraries like pyEMD. Once you have created the model, you can use it to predict the expected behavior of the motor's electrical and mechanical signals and compare it to the actual behavior to detect any anomalies.

In [ ]:
from PyEMD import EMD #pip install EMD-signal

emd = EMD()
imfs = emd(df.to_numpy())


In [96]:
np.argmax(fft_data[0])

2

Create condition indicators and features: Based on the results of your MCSA and model-based VI analyses, you can create condition indicators and features that can be used to train an ML model. For example, you might extract frequency spectrum peaks, statistical properties of the data, or time-domain features.

In [114]:
import numpy as np

peak_freqs = np.argmax(fft_data, axis=1)
mean_currents = np.mean(df, axis=1)


In [115]:
peak_freqs

array([2, 2, 2, ..., 0, 0, 0], dtype=int64)

In [116]:
mean_currents

0        1505.666667
1        1505.000000
2        1500.333333
3        1526.333333
4       10000.000000
            ...     
9995     1577.666667
9996     1580.666667
9997     1578.666667
9998     1580.333333
9999     1579.000000
Length: 10000, dtype: float64

There are various ways to detect anomalies, but one common method is to use a statistical approach based on the distribution of the feature values. For example, you could calculate the mean and standard deviation of the feature values and define a threshold for detecting anomalies based on how many standard deviations away from the mean a value is. Any feature values that fall outside this threshold would be flagged as potential anomalies.

In [117]:

# Calculate mean and standard deviation of peak frequency data
mean_freq = np.mean(mean_currents)
std_freq = np.std(mean_currents)


# Define threshold for detecting anomalies
thresh_freq = mean_freq + 3 * std_freq

# Identify potential anomalies
anomalies_freq = np.where(peak_freqs > thresh_freq)[0]


In [118]:
anomalies_freq

array([], dtype=int64)

In [75]:
sum(peak_freqs > thresh_freq)

0

In [27]:
fft_data

array([[4517.    -0.j        ,  894.5-1923.44242181j,
         894.5+1923.44242181j],
       [4515.    -0.j        ,  885. -1932.96870125j,
         885. +1932.96870125j],
       [4501.    -0.j        ,  880. -1955.48536175j,
         880. +1955.48536175j],
       ...,
       [4736.    -0.j        ,  368. -1153.54583784j,
         368. +1153.54583784j],
       [4741.    -0.j        ,  373. -1151.81378703j,
         373. +1151.81378703j],
       [4737.    -0.j        ,  384. -1136.22532977j,
         384. +1136.22532977j]])

In [43]:
peak_freqs

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [67]:
from scipy.fft import fft
import numpy as np

def get_anomalies(df):
    
    # Perform FFT on the data
    fft_data = fft(df.to_numpy(), axis=1)

    # Calculate the peak frequency for each IMFs
    peak_freqs = np.argmax(fft_data, axis=1)

    # Calculate the mean current for each row
    mean_currents = np.mean(df, axis=1)

    # Calculate the mean and standard deviation of peak frequency data
    mean_freq = np.mean(mean_currents)
    std_freq = np.std(mean_currents)

    # Define threshold for detecting anomalies
    thresh_freq = mean_freq + 3 * std_freq

    # Identify potential anomalies
    anomalies_freq = np.where(peak_freqs > thresh_freq)[0]
    
    return anomalies_freq

In [50]:
df = pd.DataFrame()

In [54]:
import pandas as pd
datafolder = os.listdir('data/s3Files/')
# df = pd.concat([pd.read_csv(f, header=None,sep=',') for f in datafolder])

for file in datafolder:
   df = pd.concat([pd.read_csv('data/s3Files/'+file,header=None,sep=',')])



In [57]:
df.drop(columns=[0,4],inplace=True)
df.columns = ['current_1', 'current_2', 'current_3']

In [59]:
get_anomalies(df)

array([], dtype=int64)

In [60]:
import copy
df1 = copy.deepcopy(df)

In [61]:
import random

# select a random row index to introduce anomalies
row_idx = random.randint(0, len(df1)-1)

# add random noise to the selected row
df1.iloc[row_idx] += np.random.normal(0, 0.1, len(df1.columns))


In [68]:
get_anomalies(df1)

array([], dtype=int64)

In [78]:
df

,current_1,current_2,current_3
0,1959,2389,106
1,1952,2391,118
2,1942,2390,124
3,1941,2393,126
4,1930,2395,126
...,...,...,...
9995,2214,1978,368
9996,2217,1985,358
9997,2218,1989,346
9998,2220,1996,337


The code seems correct. It first generates random indices to modify a specified number of values in the chunk. Then, it modifies the values at those indices by adding a random deviation. After that, it processes the modified chunk data using FFT to obtain the peak frequencies and compute the threshold. Finally, it checks for anomalies and alerts if any are detected.

However, it is important to note that the specific parameters used, such as the number of anomalies and the maximum deviation, may need to be adjusted depending on the characteristics of the data being analyzed. Additionally, it would be a good practice to split the data into training and testing sets, and only introduce anomalies in the testing set to evaluate the performance of the anomaly detection algorithm.

In [102]:
import pandas as pd
import numpy as np
from scipy.fft import fft
from sklearn.preprocessing import StandardScaler


# Define the chunk size and anomaly threshold
chunk_size = 10000  # process 10000 rows at a time
# anomaly_thresh = 0.5  # set the anomaly threshold

# Read the data in chunks
datafolder = 'data/s3Files/'
data_files = os.listdir(datafolder)

# Iterate over the files and process the data in chunks
for file in data_files:
    # Load the data chunk by chunk
    for chunk in pd.read_csv(datafolder + file, header=None, chunksize=chunk_size):


        # Generate random indices to modify
        num_anomalies = 100
        anomaly_indices = np.random.choice(chunk.index, num_anomalies, replace=False)

        # Modify the values at the anomaly indices
        max_deviation = 24
        for index in anomaly_indices:
            row = chunk.loc[index]
            col = np.random.choice(chunk.columns)
            deviation = max_deviation * np.random.random()
            chunk.loc[index, col] = row[col] + deviation
       
        scaler = StandardScaler()
        scaled_df = scaler.fit_transform(df)

        # Process the chunk data
        fft_data = fft(chunk.to_numpy(), axis=1)
        peak_freqs = np.argmax(fft_data, axis=1)
        mean_currents = np.mean(chunk, axis=1)
        mean_freq = np.mean(peak_freqs)
        std_freq = np.std(peak_freqs)
        thresh_freq = mean_freq + 3 * std_freq
        anomalies_freq = np.where(peak_freqs > thresh_freq)[0]
        # print(thresh_freq)
        
        # Check for anomalies and alert if detected
        if len(anomalies_freq) > 0:
            print(f"Anomaly detected in {file}: {len(anomalies_freq)} anomalies detected in chunk {chunk.index[0]} - {chunk.index[-1]}")
